In [ ]:
%reload_ext autoreload
%autoreload 2

## New fastai

In [ ]:
from nb_007b import *

In [ ]:
CLAS_PATH = Path('../data/aclImdb/clas/')

It's assumed that CLAS_PATH contains the files itos.pkl, trian_ids.npy, valid_ids.npy, trian_lbl.npy and valid_lbl.npy

In [ ]:
train_ds, valid_ds = TextDataset.from_ids(CLAS_PATH)

In [ ]:
bs,bptt = 48,70

In [ ]:
train_sampler = SortSampler(train_ds.ids, key=lambda x: len(train_ds.ids[x]))
valid_sampler = SortSampler(valid_ds.ids, key=lambda x: len(valid_ds.ids[x]))
train_dl = DeviceDataLoader.create(train_ds, bs//2, sampler=train_sampler, collate_fn=pad_collate)
valid_dl = DeviceDataLoader.create(valid_ds, bs,  sampler=valid_sampler, collate_fn=pad_collate)
data = DataBunch(train_dl, valid_dl)

In [ ]:
vocab_size,n_class = len(train_ds.vocab.itos),2
emb_sz,nh,nl = 400,1150,3
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.5

In [ ]:
model = get_rnn_classifier(bptt, 20*70, n_class, vocab_size, emb_sz=emb_sz, n_hid=nh, n_layers=nl, pad_token=1,
          layers=[emb_sz*3, 50, n_class], drops=[dps[4], 0.1],
          input_p=dps[0], weight_p=dps[1], embed_p=dps[2], hidden_p=dps[3])

Uncomment to create a model.

In [ ]:
#torch.save(model[0].state_dict(), 'models/body.pth')
#torch.save(model[1].state_dict(), 'models/head.pth')

In [ ]:
groups = [nn.Sequential(model[0].encoder, model[0].encoder_dp)]
groups += [nn.Sequential(rnn, dp) for rnn, dp in zip(model[0].rnns, model[0].hidden_dps)] 
groups.append(model[1])

In [ ]:
learn = Learner(data, model)
learn.layer_groups = groups
learn.callbacks.append(RNNTrainer(learn, bptt, alpha=2, beta=1, adjust=False))
learn.callback_fns.append(partial(GradientClipping, clip=0.12))
learn.metrics = [accuracy]
learn.opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learn.freeze()

In [ ]:
learn.model[0].load_state_dict(torch.load('models/body.pth'))
learn.model[1].load_state_dict(torch.load('models/head.pth'))

In [ ]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.manual_seed(7)
torch.cuda.manual_seed(7)

In [ ]:
learn.fit(1, 1e-2, wd=0)

In [ ]:
learn.recorder.losses[:10]

## Old fastai

Restart here

In [ ]:
from fastai.text import *

In [ ]:
CLAS_PATH = Path('../data/aclImdb/clas/')

In [ ]:
itos = pickle.load(open(CLAS_PATH/'tmp'/'itos.pkl', 'rb'))
trn_clas = np.load(CLAS_PATH/'tmp'/'train_ids.npy')
val_clas = np.load(CLAS_PATH/'tmp'/'valid_ids.npy')
trn_labels = np.load(CLAS_PATH/'tmp'/'train_lbl.npy')
val_labels = np.load(CLAS_PATH/'tmp'/'valid_lbl.npy')

In [ ]:
bptt,em_sz,nh,nl = 70,400,1150,3
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 48
wd=1e-7

In [ ]:
min_lbl = trn_labels.min()
trn_labels -= min_lbl
val_labels -= min_lbl
c=int(trn_labels.max())+1

In [ ]:
trn_ds = TextDataset(trn_clas, trn_labels)
val_ds = TextDataset(val_clas, val_labels)
trn_samp = SortSampler(trn_clas, key=lambda x: len(trn_clas[x]))
val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
md = ModelData(CLAS_PATH, trn_dl, val_dl)

In [ ]:
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.5

In [ ]:
m = get_rnn_classifer(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
          layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
          dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

In [ ]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [ ]:
learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learn.clip=0.12
learn.metrics = [accuracy]

In [ ]:
MODEL_PATH = Path('models/')
body = torch.load(MODEL_PATH/'body.pth', map_location='cuda')
head = torch.load(MODEL_PATH/'head.pth', map_location='cuda')

In [ ]:
replace = {'encoder_dp.emb.weight':'encoder_with_dropout.embed.weight'}
for i in range(3): replace[f'rnns.{i}.weight_hh_l0_raw'] = f'rnns.{i}.module.weight_hh_l0_raw'
to_del = [f'rnns.{i}.module.weight_hh_l0' for i in range(3)]
new_body = {}
for n,p in body.items():
    n1 = replace[n] if n in replace else n
    if not n in to_del: new_body[n1] = model[n]

In [ ]:
replace = {'layers.2.weight':'layers.0.lin.weight',
           'layers.2.bias':'layers.0.lin.bias',
           'layers.0.weight':'layers.0.bn.weight',
           'layers.0.bias':'layers.0.bn.bias',
           'layers.0.running_mean':'layers.0.bn.running_mean',
           'layers.0.running_var':'layers.0.bn.running_var',
           'layers.6.weight':'layers.1.lin.weight',
           'layers.6.bias':'layers.1.lin.bias',
           'layers.4.weight':'layers.1.bn.weight',
           'layers.4.bias':'layers.1.bn.bias',
           'layers.4.running_mean':'layers.1.bn.running_mean',
           'layers.4.running_var':'layers.1.bn.running_var',
           }
new_head = {}
for n,p in head.items():
    if n in replace: new_head[replace[n]] = p.clone()

In [ ]:
m[0].load_state_dict(new_body)
m[1].load_state_dict(new_head)

In [ ]:
learn.freeze_to(-1)

In [ ]:
lr,wd=1e-2,0

In [ ]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.manual_seed(7)
torch.cuda.manual_seed(7)

In [ ]:
learn.fit(lr, 1, wds=wd)

In [ ]:
learn.sched.losses[:10]